## Installation de JAGS et importation des données

In [1]:
system("apt-get update -y && apt-get install -y jags")
install.packages('runjags')

In [2]:
# Charger les bibliothèques nécessaires
library(dplyr)
library(ggplot2)
library(tidyverse)
library(runjags)

# Modèle génératif et espace des paramètres

On veut construire un modèle prédictif pour le salaire des individus pour lesquels on connait 11 variables applicatives (âge, éducation, experience professionnelle, etc.), 434 individus dans un jeu de données d'entraînement et 100 individus dans le jeu de données de test.

On note $Y_i \in \mathbb{R}$ le salaire de l'individu $i$ et $X_i \in \mathbb{R}^{11}$ les variables explicatives de l'individu $i$.

Régression linéaire:
$Y_i = \beta_0 + \beta_1 X_{1i} + \beta_2 X_{2i} + \cdots + \beta_{10} X_{10i} + \beta_{11} X_{11i} + \sigma \epsilon_i$
12 paramètres $\beta = (\beta_0, \beta_1, \beta_2, \cdots, \beta_{10}, \beta_{11})$  $\epsilon_i \sim \mathcal{N}(0, 1)$ et $\sigma \in \mathbb{R^+}$

Certaines variables peuvent avoir peu ou pas d'effet sur le salaire, ie des $\beta_i = 0$.

## Distribution à postériori

A partir de $y=(y_1, y_2, \cdots, y_{434})$ et $ X=(X_1, X_2, \cdots, X_{434})$, on peut calculer $P(\beta | y, X)$ et regarder $P(\beta_j \neq 0 | y, X)$ pour chaque coefficient.

## Performances prédictives

Un manière d'évaluer la performance prédictive $\hat{\beta}_{bayes} = E(\beta | y, X)$
$X_{test}$ matrice 100x11
$y_{test}$ les observations
$y_{test,Bayes} = X_{test} \hat{\beta}_{bayes}$

# Degrès de croyance

Que croit-on sur l'effet de chaque variable explicative sur le salaire?

Nous pouvons formuler des hypothèse à priori sur l'impact des différentes variables :

- Éducation : On s’attend à une relation positive entre le niveau d’éducation et le salaire.

- Expérience professionnelle (workexp) : Plus d’expérience est généralement associée à un salaire plus élevé.

- Sexe (female) : Il est bien documenté qu’un écart salarial entre hommes et femmes existe (gender pay gap).

- Région (south) : Il peut y avoir des différences salariales régionales.

- Appartenance syndicale (unionmember) : Être syndiqué peut influencer le salaire.

- Âge (age) : En général, l’âge est corrélé à l’expérience professionnelle, mais il pourrait aussi capturer d’autres dynamiques comme la discrimination liée à l’âge.

- Ethnicité (ethnicity) : Certaines inégalités salariales peuvent exister selon l’origine ethnique.

- Occupation et secteur (occupation, sector) : Le type d’emploi et le secteur d’activité influencent fortement le salaire.

- Statut matrimonial (married) : Certaines études suggèrent que les personnes mariées ont des salaires plus élevés, en particulier les hommes.


### Estimation d'un paramètre $\theta$ avec des données

Ayant obtenu {$Y=y$} , $P(\theta | y) = \frac{P(\theta , y)}{P(y)} = \frac{P(\theta) P(y | \theta)}{P(y)}$

Donc $posterior \propto prior \times likelihood$

$prior = P(\theta)$ : croyance initial avant d'observer les données
$likelihood = P(y | \theta)$ : Probabilité d'observer les données en fonction des paramètres du modèle
$posterior = P(\theta | y)$ : La Distribution à posteriori après avoir observé les données



# 2. Observation

Nous allons commencer par importer les données.

In [3]:
wage_data = read.csv("https://raw.githubusercontent.com/derghalmanal/Wage/refs/heads/main/wage_data.csv", header=TRUE, sep=",")

Notre jeu de données comprend 534 individus. Chaque individu a 11 variables : education, south, female, workexp, unionmember, wages, age, ethnicity, occupation, sector, married.

In [4]:
#shape of the data
dim(wage_data)

- Variables quantitatives:
    > **education**: Années d'études
    >**workexp**: Années d'expêrience professionnelle
    >**wages**: Salaire en dollars horaires
    >**age**: âge

- Variables qualitatives:
    >**south**: Vit dans le sud ou non
    >**female**: Indicateur de genre (0 = Homme, 1 = Femme)
    >**unionmember**: Indicateur de membre du syndicat (0 = Non, 1 = Oui)
    >**married**: Indicateur de statut marital. Marié(e) ou non.

- Variables categorielles:
    >**ethnicity**: Ethnicité (White, Hispanic, Other)
    >**occupation**: Occupation (Management, Sales, Clerical, Professional, Service, Other)
    >**sector**: Secteur d'activité (Manufactoring, Construction, Other)


In [5]:
head(wage_data, 5)

In [6]:
str(wage_data)

## Visualisation

Distribution des salaires

In [7]:
# Histogramme des salaires
ggplot(wage_data, aes(x=wages)) +
  geom_histogram(binwidth = 1, fill="skyblue", color="black", alpha=0.7) +
  labs(title="Distribution des Salaires", x="Salaire", y="Fréquence")

Salaire par sexe (female)

In [8]:
# Boxplot des salaires par sexe
ggplot(wage_data, aes(x=factor(female), y=wages, fill=factor(female))) +
  geom_boxplot() +
  labs(title="Salaires par Sexe", x="Sexe (0 = Homme, 1 = Femme)", y="Salaire") +
  scale_fill_manual(values=c("lightblue", "lightcoral"))


Salaire par ethnie

In [9]:
# Boxplot des salaires par ethnie
ggplot(wage_data, aes(x=ethnicity, y=wages, fill=ethnicity)) +
  geom_boxplot() +
  labs(title="Salaires par Ethnie", x="Ethnie", y="Salaire") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

Relation entre experience professionnelle et salaires

In [10]:
# Nuage de points (scatter plot) entre expérience et salaire
ggplot(wage_data, aes(x=workexp, y=wages)) +
  geom_point(alpha=0.5, color="darkblue") +
  labs(title="Relation entre Expérience et Salaire", x="Expérience (années)", y="Salaire")

Salaire par secteur

In [11]:
# Boxplot des salaires par secteur
ggplot(wage_data, aes(x=sector, y=wages, fill=sector)) +
  geom_boxplot() +
  labs(title="Salaires par Secteur", x="Secteur", y="Salaire") +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

Relation entre éducation et salaire (education vs wages)
On observe que plus l'éducation est élevée, plus le salaire est haut

In [12]:
# Nuage de points entre éducation et salaire
ggplot(wage_data, aes(x=education, y=wages)) +
  geom_point(alpha=0.5, color="purple") +
  labs(title="Relation entre Éducation et Salaire", x="Niveau d'Éducation", y="Salaire")

 Répartition des salaires selon le mariage (married).
 Ici on observe que les personnes mariées ont un salaire plus haut

In [13]:
# Boxplot des salaires selon le statut marital
ggplot(wage_data, aes(x=factor(married), y=wages, fill=factor(married))) +
  geom_boxplot() +
  labs(title="Salaires par Statut Marital", x="Statut Marital (0 = Non Marié, 1 = Marié)", y="Salaire") +
  scale_fill_manual(values=c("lightgreen", "lightpink"))

Matrice de corrélation (pour les variables numériques)
Ici on observe les correlations entre les variables. Il y a une forte corrélation entre l'age et l'experience professionnelle. Il existe une corrélation également entre le salaire et le niveau d'éducation.
Il y a également une corrélation négative entre le niveau d'éducation et l'experience professionnelle

In [14]:
# Matrice de corrélation
install.packages("corrplot")
library(corrplot)
cor_matrix <- cor(wage_data[, c("education", "workexp", "wages", "age")], use="complete.obs")
corrplot(cor_matrix, method="circle", type="lower", tl.cex=0.8)

Relation entre l'age et l'experience professionnelle
On observe que l'age des individus est fortement correle avec l'experience professionnelle, ce qui fait sens puisque plus l'age est grand, plus l'experience professionnelle est longue.

In [15]:
# Nuage de points entre éducation et salaire
ggplot(wage_data, aes(x=age, y=workexp)) +
  geom_point(alpha=0.5, color="purple") +
  labs(title="Relation entre Age et Expérience Professionnelle", x="Age", y="Expérieence Professionnelle")

In [16]:
# Pairplot avec GGally
library(GGally)
ggpairs(wage_data[, c("education", "workexp", "wages", "age")])

On réalise un histogramme pour chaque variable.

In [17]:
numeric_columns <- sapply(wage_data, is.numeric) # Identifier les colonnes numériques
numeric_data <- wage_data[, numeric_columns] # Extraire les colonnes numériques

# Créer les histogrammes pour chaque colonne numérique
par(mfrow=c(3,3)) 
for (col in colnames(numeric_data)) {
  hist(numeric_data[[col]], 
       main=paste("Histogramme de", col), 
       xlab=col, 
       col="skyblue", 
       border="black", 
       breaks=60)
}

par(mfrow=c(1,1))


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=650b0311-43bb-4704-b320-0c62dbd2dedc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>